In [1]:
import numpy as np

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [3]:
x = np.load('x.npy') # 5000 images
y = np.load('y.npy')

theta_0 = np.load('theta_0.npy')
theta_1 = np.load('theta_1.npy')

In [4]:
# Step 1. Convert Pixels Using Fixed Point Notation
imgIndex = 0
image = x[imgIndex]

# Step 2. Convert First Hidden Layer Weights Into Fixed Point Notation
xp = np.hstack((1, image))
hiddenLayer1 = np.dot(xp,theta_0.T)

# Step 3. Make The Second Hidden Layer Multiplication X*W 
fl_hiddenLayer1 = []
for i in range(0, len(theta_0)): # [0, 24]
    row = theta_0[i]
    product = 0
    for j in range(0, len(row)): # [0, 400]
        weight = row[j]
        pixel = xp[j]
        # todo convertir les valeurs en nombres de 4 bits signés?
        product = product + weight * pixel
    fl_hiddenLayer1.append(product)

for element in fl_hiddenLayer1:
    print(element)

# Step 4. Apply the Sigmoid Function For First Hidden Layer

# Step 5. Convert Second Hidden Layer Weights Into Fixed Point Notation

# Step 6. Make The Second Hidden Layer Multiplication X*W 

# Step 7. Apply the Sigmoid Function For Second Hidden Layer

# Step 8. Display FPGA Approximation


-2.8864903707584992
-2.627589487865158
4.978249302788464
0.13652241097741863
0.9344568207070681
5.649162795307462
-4.574623497386452
-6.436168853406793
5.570101027535296
-3.894766638919115
-2.521838093517566
-3.7859793619873496
-2.9884101130989316
-2.927813861994206
-5.4667274936400885
2.0176854495123466
4.732885339700824
-2.3207866950517317
1.6731172507068153
1.36724315335429
-3.673168770120536
4.103596326223249
3.670360437342782
2.845977233877717
-2.169111998038491


In [ ]:
# Calculer sur le FPGA
xp = np.hstack((1, image)) # Ajouter 1 pour le biais
hiddenLayer1 = np.dot(xp,theta_0.T)

a = sigmoid(hiddenLayer1) # Sortie de la couche 1
ap = np.hstack((1, a)) # Ajouter 1 pour le biais
b = sigmoid(np.dot(ap,theta_1.T)) # Sortie de la couche 2
print("FPGA Output:",b)

# Calculer dans le notebook
pred = b.argmax()
print("Prediction:", pred, "Label:", y[idx])